In [54]:
# Import libraries
import numpy as np
import pandas as pd
from keras.layers import Dropout
import os
import cv2
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import keras
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn import svm
from sklearn.externals import joblib
%matplotlib inline

In [55]:
# load files
train = pd.read_csv(r'/Users/Bhargavi/Desktop/ML_project/Dataset/train.csv')
test = pd.read_csv(r'/Users/Bhargavi/Desktop/ML_project/Dataset/test.csv')

In [56]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [60]:
# set path to read train and test image
TRAIN_PATH = '/Users/Bhargavi/Desktop/ML_project/Dataset/train_img/'
TEST_PATH = '/Users/Bhargavi/Desktop/ML_project/Dataset/test_img/'

In [61]:
# function to read images as arrays
def read_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128,128)) # you can resize to  (128,128) or (256,256)
    return img

In [62]:
# Read and store images
train_data = []
test_data = []
train_labels = train['label'].values

for img in tqdm(train['image_id'].values):
    train_data.append(read_image(TRAIN_PATH + '{}.png'.format(img)))

train_array = np.asarray(train_data)
    
for img in tqdm(test['image_id'].values):
    test_data.append(read_image(TEST_PATH + '{}.png'.format(img)))

test_array = np.asarray(test_data)
    
train_array = train_array.reshape(train_array.shape[0], 128, 128, 3).astype('float32')
test_array = test_array.reshape(test_array.shape[0], 128, 128, 3).astype('float32')

train_array /= 255
test_array /= 255
print(train_array.shape)
print(test_array.shape)

100%|██████████| 965/965 [00:03<00:00, 241.89it/s]


(2250, 128, 128, 3)
(965, 128, 128, 3)


In [63]:
# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]

label_list = test['label'].tolist()
Y_test = {k:v+1 for v,k in enumerate(set(label_list))}
y_test = [Y_test[k] for k in label_list]

In [64]:
num_classes = 26
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [65]:
print(train_array.shape)
print(y_train.shape)
print(test_array.shape)
print(y_test.shape)

(2250, 128, 128, 3)
(2250, 26)
(965, 128, 128, 3)
(965, 26)


In [99]:
# Random Forest model creation 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

x_train = np.reshape(train_array, (train_array.shape[0], -1))
x_test = np.reshape(test_array, (test_array.shape[0], -1))
print(x_train.shape)
print(x_test.shape)
clf = RandomForestClassifier(n_estimators=50, max_depth=None,min_samples_split=20, random_state=1).fit(x_train,y_train)
Results=clf.predict(x_test)

# Classification report for RF 
print (classification_report(Results,y_test))

(2250, 49152)
(965, 49152)
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       0.00      0.00      0.00         0
          5       0.00      0.00      0.00         0
          6       0.00      0.00      0.00         0
          7       0.00      0.00      0.00         0
          8       0.00      0.00      0.00         0
          9       0.00      0.00      0.00         0
         10       0.00      0.00      0.00         0
         11       0.00      0.00      0.00         0
         12       0.00      0.00      0.00         0
         13       0.00      0.00      0.00         0
         14       0.00      0.00      0.00         0
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         0
         17       

/Users/Bhargavi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Bhargavi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [98]:
# Random Forest cross validation and accuracy scores

from sklearn import model_selection
clf = RandomForestClassifier(n_estimators=50, max_depth=None,min_samples_split=20, random_state=1).fit(x_train,y_train)
results = model_selection.cross_val_score(clf, x_train,y_train, cv=5)
print (results.mean())

0.000888888888889
